In [19]:
import requests
import threading
import random
import logging
import json
import time
from bs4 import BeautifulSoup

In [2]:
class UserAgent(object):
    _instance_lock = threading.Lock()
    
    def __init__(self):
        self.__agents_pool = list()
        with open('./UserAgents/useragents.txt','r') as read_ob:
            for line in read_ob.readlines():
                self.__agents_pool.append(line.strip())
    
    def __new__(cls, *args, **kwargs):
        if not hasattr(UserAgent, "_instance"):
            with UserAgent._instance_lock:
                if not hasattr(UserAgent, "_instance"): 
                    UserAgent._instance = object.__new__(cls)
        return UserAgent._instance
    
    def get_useragent_randomly(self):
        return random.choice(self.__agents_pool)
    

class Crawl():

    def __init__(self):
        self.__session = None
        self.__headers = {
            'User-Agent':UserAgent().get_useragent_randomly(),
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language':'zh-cn',
        }
        self.__proxies = [
            {"http":"112.111.217.114:9999"},
            {"http":"180.118.128.118:9000"},
            {"http":"171.11.29.217:9999"},
            {"http":"120.83.109.191:9999"}
        ]
        
    def request_get(self, url, **kwargs):
        logging.info('scraping {}...'.format(url))
        try:
            response = requests.get(url, headers=self.__headers, proxies=random.choice(self.__proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, self.__url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
     
    def request_post(self, url, **kwargs):
        logging.info('scraping {}...'.format(url))
        try:
            response = requests.post(url, headers=self.__headers, proxies=random.choice(self.__proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, self.__url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
    
    def session_get(self, url, **kwargs):
        self.check_session()
        logging.info('scraping {}...'.format(url))
        try:
            response = self.__session.get(url, headers=self.__headers, **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, self.__url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
    
    def session_post(self, url, **kwargs):
        self.check_session()
        logging.info('scraping {}...'.format(url))
        try:
            response = self.__session.post(url, headers=self.__headers, **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, self.__url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url)) 

    def get_session(self):
        return self.__session
    
    def set_session(self, session):
        self.__session = session
        return True
    
    def save_cookies(self, user):
        cookieJar = requests.cookies.RequestsCookieJar()
        for cookie in user.cookies:
            cookieJar.set(cookie.name,cookie.value)
        for cookie in user.headers['Set-Cookie'].split(";"):
            key = cookie.split('=')[0]
            value = cookie.split('=')[1]
            cookieJar.set(key,value)
        return cookieJar
    
    def check_session(self):
        if self.__session == None:
            self.__session = requests.session()
        return
    
    def add_header(self, headers):
        for key, value in headers.items():
            self.__headers[key] = value


In [13]:
class DoubanCrawl(Crawl):
    
    def __init__(self):
        super(DoubanCrawl, self).__init__()
#         self.__user = None
#         self._cookies = None
        self.__login()
    
    def __login(self):
        post_data = {
            'name':'18664678368',
            'password':'LJC970412',
            'remember':'false'
        }
        self.add_header({
            "Referer":'https://accounts.douban.com/passport/login'
        })
        user = self.session_post('https://accounts.douban.com/j/mobile/login/basic', data=post_data)
        login_detail = json.loads(user.text)
        if login_detail['status'] == 'success':
            print('login success!')
#             self.__cookies = self.save_cookies(user)
        else:
            print('login failed!')
        return user.text
        
    def search(self, query, cat=''):
        res = dict()
        params = {
            'q':query,
            'cat':cat
        }
        count=0
        response = self.session_get('https://www.douban.com/search', params=params)
        soup = BeautifulSoup(response.content, 'html.parser')
        results = soup.find(class_='result-list').find_all(class_='result')
        for result in results:
            pic = result.find(class_='pic')
            content = result.find(class_='content')
            img = pic.img.get('src')
            link = content.a.get('href')
            name = content.a.text
            description = content.p.text if content.p else ''
            res[count]={
                'name':name,
                'img':img,
                'link':link,
                'description':description
            }
            count+=1
        return res

In [23]:
class DoubanMovie():
    
    def get_comment(self, url=None, movie_id=None, page_max=50):
        movie_url = None
#         if movie_name is not None:
            
        if movie_id is not None:
            movie_url = 'https://movie.douban.com/subject/{}/comments'.format(str(movie_id))
        else:
            movie_url = url
        count = 0
        for page in range(page_max+1):
            print("开始爬取第{0}页***********************************************************************：".format(page+1))
            params={
                'start':page*20,
                'limit':20,
                'status':'P',
                'sort':'new_score'
            }
            html = self.session_get(movie_url, params=params)
            print(html.url)
            soup = BeautifulSoup(html.content, 'html.parser')
            comments = soup.find(id='comments').find_all(class_='comment-item')
            if len(comments) > 1:
                for comment in comments:
                    content = comment.find(class_='comment-content').text
                    user_name = comment.find(class_='comment-info').a.text
                    comment_time = comment.find(class_='comment-info').find(class_='comment-time').attrs['title']
                    print('comment:{}, user_name:{}, comment_time:{}'.format(content,user_name,comment_time))
                    count += 1
                print("count={}".format(count))
                time.sleep(int(random.choice([0.5, 0.2, 0.3])))
            else:
                print("大约共{0}页评论".format(page+1))
                break
            
#     def get_review_by_url(self, url , page_max=50):
#         res = dict()
#         self.set_url(url)
#         count = 0
#         for page in range(page_max+1):
#             print("开始爬取第{0}页***********************************************************************：".format(page+1))
#             self.set_params({
#                 'start':page*20
#             })
#             html = self.session_get()
#             soup = BeautifulSoup(html.content, 'html.parser')
#             reviews = soup.find(class_='review-list').find_all(class_='review-item')
#             for review in reviews:
#                 user = review.find(class_='main-hd')
#                 user_name = user.find(class_='name').text
#                 user_link = user.find(class_='name').a.get(href)
                
#                 content = review.find(class_='main-bd')
#                 title = content.find('h2').text
#                 link = content.find('h2').a.get('href')
#                 self.set_url(link)
#                 review_response = self.request_get()
#                 review_soup = BeautifulSoup(review_response, 'html.parser')
#                 article = review_soup.find(class_='article')
#                 review_content = article.find(class_='review-content').text
#                 res[title] = {
#                     'user_name':user_name,
#                     'user_link':user_link,
#                     'link':link,
#                     'review_content':review_content
#                 }
#             return res
        
#     def get_review_by_id(self, movie_id, page_max=50):
#         url = 'https://movie.douban.com/subject/{}/reviews'.format(str(movie_id))
#         self.get_review_by_url(url, page_max)
    
    def search_movie(self, movie_name):
        result = self.search(movie_name, cat='1002')
        return result

In [15]:
crawl = DoubanMovie()

login success!


In [21]:
res = crawl.search('拆弹专家')

In [22]:
res

{0: {'name': '拆弹专家 ',
  'img': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2465858820.jpg',
  'link': 'https://www.douban.com/link2/?url=https%3A%2F%2Fmovie.douban.com%2Fsubject%2F26748673%2F&query=%E6%8B%86%E5%BC%B9%E4%B8%93%E5%AE%B6&cat_id=1002&type=search&pos=0',
  'description': '章在山（刘德华 饰）是香港警队“爆炸品处理科”的一名高级督察。七年前，他潜伏到头号通缉犯火爆（姜武 饰）的犯罪团伙中，在一次打劫金库的行动中，章在山表露了其拆...'},
 1: {'name': '拆弹专家2 ',
  'img': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2621379901.jpg',
  'link': 'https://www.douban.com/link2/?url=https%3A%2F%2Fmovie.douban.com%2Fsubject%2F30171424%2F&query=%E6%8B%86%E5%BC%B9%E4%B8%93%E5%AE%B6&cat_id=1002&type=search&pos=1',
  'description': '香港某处发生爆炸案，前拆弹专家潘乘风（刘德华 饰）因昏迷于现场，被警方怀疑牵涉其中。苏醒后的潘乘风只能一边逃亡一边查明真相，然而，他的好友董卓文（刘青云 饰）...'},
 2: {'name': '拆弹专家宝贝炸弹 ',
  'img': 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2164846726.jpg',
  'link': 'https://www.douban.com/link2/?url=https%3A%2F%2Fmovie.douban.com%2Fsubject%2F1298391%2F&query=%E6%8

In [20]:
crawl.get_comment('https://movie.douban.com/subject/30171424/comments')

开始爬取第1页***********************************************************************：
https://movie.douban.com/subject/30171424/comments?start=0&limit=20&status=P&sort=new_score
comment:
这哪里是拆弹2这是无间道4……各路反转顺滑到不行。BUG这么少的片子近期很难得了。除了有明显反社会人格的人如何被放心诏安以外，就是倪妮太美，其他都还算经得起推敲。节奏也是非常好，不会疲乏，怎么看都觉得带劲。算是院线港片天花板了吧。
, user_name:一口吃掉小蛋糕, comment_time:2020-12-23 23:58:05
comment:
在当下这个语境里，敢把整个香港炸翻的也只有邱礼涛了。同样是香港导演，同样塑造英雄，邱礼涛和林超贤简直形成了两种极端，一个奏挽歌一个奏颂歌。《拆弹专家2》拍的是被集体反复利用后抛弃的个体，是制度的弃子，充满悲情，《紧急救援》拍的是大无畏的英雄主义，是为了人民安全可以不顾家人和自己、一股脑冲锋陷阵的救人机器。邱礼涛还是香港电影的打工人，仍然能对个体命运投注悲悯和同情，而林超贤已经沉迷在主旋律叙事当中了，只能用“国旗下的讲话”来包装出一个个教科书式的标兵。
, user_name:一口大井子, comment_time:2020-12-24 03:06:13
comment:
还是推荐的，大场面和枪战音效的影院效果不错，但不吐不快：角色动机和转变都极为生硬，说白了故事不是剧情推进，而是导演推进，全员工具人，倪妮角色（谎言）戏份应该是刘青云来完成，这才有因果救赎的宿命感，如今刘青云存在感太低了。邱礼涛量级最大的行活儿，手快精良，不一定好但必须有自我表达：“社会学家”又拿出了当年拍《的士判官》那股劲头，时代不同了，他不再不挥舞屠刀改玩炸弹。PS，刘德华太帅了！第一集自爆的小哥这次又爆了
, user_name:乌鸦火堂, comment_time:2020-12-24 11:49:39
comment:
是晁盖也是宋江，是警察也是匪徒，是英雄也是疯子，是解铃人也是系铃人。为何不认识她还是会心动，不记得他还是会心痛，为何要在用人即弃的世界苟且，在万夫莫开的关口坚守。为了拯

https://movie.douban.com/subject/30171424/comments?start=40&limit=20&status=P&sort=new_score
comment:
这方方面面的，香港年轻导演学不会啊，只会拍一些舔金马和欧洲影节的纪录片是没用的，牛逼你就拍一部让大陆几亿观众看了能更理解香港的。
, user_name:黑狗成, comment_time:2020-12-25 16:16:01
comment:
香港的无根性啊……选择记忆即选择体制，无论怎么选择，最后的核弹都会引爆，巨浪还是拍打香江岸头，刘德华还是要……电影里失忆这样的小概率事件左右了结局，未来的现实呢？只有深谙极左之道的邱礼涛能掌控这样的题材，刘德华的角色依然有黄秋生（叉烧包、伊波拉）那些无政府主义的影子，2020年华语片最佳！好多小细节特别值得玩味，比如最后是倪妮指挥香港各个政府机关部门…
, user_name:日灼, comment_time:2020-12-24 13:48:30
comment:
牛逼啊牛逼！“置之死地而幸存，陷之绝境而复生”。30年过去了，华语电影终于出现了一部超越林岭东的《圣战风云》的反恐片（which means超越了诺兰以及全世界除了《盗日者》之外的同题材电影）。正看是个好人误入歧途找回天良大破乱党的故事。反看则是忠良受辱逼上梁山最后看在恋人面子上放这世界一马的反社会传奇。白衣白头的谢君豪与黑衣华仔对撞坠海的一刹那，“兄弟情”是真是假已不重要，冲天的水柱是一腔怒火对不公制度的怒吼，完爆那啥《黑暗骑士崛起》。“失忆者颠覆自己策划的阴谋”翻新烂桥，争相重塑记忆的警匪无分正邪。华仔的表演也终于不再是电影的短板，除了做作的肢体语言恰好符合人物偏执的性格之外，眼神里终于有了糅合了受伤、愤懑和隐忍的复杂性。如此杰作，细节上的瑕疵不值一提，且看且珍惜。
, user_name:风间隼, comment_time:2020-12-24 16:09:26
comment:
3.5星。比第一部好太多了，首先，爆炸场面特效、音效就做得挺精彩的，挺震撼的；其次，刘德华、刘青云和倪妮的演技都还不错，最出色的还是刘德华的表演，比在《热血合唱团》的好很多，能演出角色的偏执、愤怒和复杂性，（刘德华快六十岁了，姜皓文也五十多了，他俩你追我跑的追逐戏真的很拼，还在

https://movie.douban.com/subject/30171424/comments?start=80&limit=20&status=P&sort=new_score
comment:
看邱礼涛的电影，不往政治隐喻想是不可能的。
这片在讲什么？
讲一个为体制服务后被放逐的人，成了反体制先锋，后又经历失忆、洗脑，重回体制，并最终为体制献身的故事。
非常邱礼涛，当然，大家看得爽就好。
最终，倪妮删除了所有记忆，只留下美好的爱情片段，你瞧，多会删减呀。
, user_name:子戈, comment_time:2020-12-25 21:35:01
comment:
信我，你是警察，不信，你是恐怖分子。哈哈哈哈。
, user_name:SydneyCarton, comment_time:2020-12-25 20:08:37
comment:
香港电影有内地演员参与就是败笔！不过也是没办法的事
, user_name:笙笙, comment_time:2020-11-30 18:21:10
comment:
刘德华和刘青云在车里坐一起立马就有港味了，仿佛看了一部迟到20年的《暗战2》。有张有弛，节奏紧凑无尿点，故事完整bug槽点都少，现实宏大，很难得。刘德华角色前半程塑造挺好，后面显得生硬。但这个角色还是挺让人动容。谁能想到，2020年，撑起港片门面的还是那两个加起来快120岁的人。
, user_name:女神的秋裤, comment_time:2020-12-25 00:23:25
comment:
太好看了，主创全程演技在线，颜值能打，想不投入都不行，倪妮气场真的太强了，不愧是气质型美女，配上激烈的枪战，紧张的拆弹剧情，有种身临其境的感觉。
, user_name:monica喵, comment_time:2020-12-24 10:16:21
comment:
一出香港的末世预言，炸完隧道炸机场，香港变孤岛，大场面的想象力足以让人讶异，文本的复杂性和对现实的观照度更是让人惊掉下巴。清除和植入记忆是对“身份认同”母题书写的一体两面，一部怎么解读都觉得还不够的作品。希望这片能大卖，邱礼涛值得。另外，《拆弹专家2》里的刘德华就是《伊波拉病毒》里的黄秋生。下一届金像奖影帝，全世界我第一个说。
, user_name:朝阳区妻夫木聪, comment

https://movie.douban.com/subject/30171424/comments?start=140&limit=20&status=P&sort=new_score
comment:
说不好看的我真不知道你们对于这种商业电影你们到底还想看什么，爆米花电影的特点就是 牺牲少部分可以忽略的逻辑错误来营造肾上腺素飙升的刺激感，这电影完美符合。血花飞溅的枪战、警匪间的博弈、正义的举动、一流的演技和超震撼的爆炸场面，我甚至觉得这已经是最好的邱礼涛，除了某些炫特效炫镜头过于冗长外几乎挑不出缺点，倪妮也美！这是一次非常不错的观影体验！PS刘德华这个角色有点冲奖的意思！
, user_name:黄悦_, comment_time:2020-12-27 11:31:10
comment:
献给2019年的香港
, user_name:堤, comment_time:2020-12-24 17:18:42
comment:
超乎预期的好看，节奏明快，情节激燃，不管是从动作角度还是场面调度都无可挑剔。刘德华+刘青云的双影帝组合为本片加色不少，倪妮不用多说，够美够媚已经足以。值得一提的是，本来报着看爽片的心去的，没想到在电影结尾居然红了眼眶，导演在整体故事节奏把控上绝对一流。《拆弹专家2》不愧是今年贺岁档最有票房潜力的电影！
, user_name:李铁柱柱柱柱, comment_time:2020-12-17 22:15:55
comment:
从以暴制暴，到以爆制爆。全片最大的悬念和意义既不是生死，也不是胜负，更不是所谓的正邪，而是选择。社会的”病“催生个体的”痛“，这几乎是每个成年人在生活中都曾有过的感受，也自然成为本片最能引发共情之处，在遗忘了切肤之“痛”的情况下，潘乘风以旁观者的视角审视周遭一切，“重建”世界观，做出最终的选择，角色自身的层次感和整个文本的丰富性在同类型中实属罕见。影片技术层面极为扎实，节奏、剪辑、调度、表演无一不是上佳，更难能可贵的是在成熟商业片外壳下包裹着复杂沉重的思想内核，多处台词和细节背后的深意令人不寒而栗，无论从哪个角度来讲，都无疑是今年最值得走进影院的国产片之一。它是近些年越发鲜见的又一次港片回魂，是邱礼涛导演集大成之作，是“双刘”在烂片泥潭里的华丽重生，也是倪妮自首秀后唯一能拿得出手的作品。
, user_name:白发生, c

https://movie.douban.com/subject/30171424/comments?start=200&limit=20&status=P&sort=new_score
comment:
三星半。依旧是不擅长叙事的邱礼涛，几个角度的回忆线其实挺生硬的，独特的还是他长久以来无所顾忌的笔法，只不过审查之下突破尺度的方式发生了转移，但他的那种没有后顾之忧的态度真是少见。最后还玩起了一把情怀，联系时局和主角的反叛，有点余味。
, user_name:Oasis, comment_time:2020-12-23 22:16:57
comment:
是体制逼疯良好市民，又还是心痛市民偏激妖魔化体制？“记者”邱礼涛以紧张刺激的商业化大片为载体，将种种当代香港残片揉碎，重新缝合包装，成就精彩寓言。哪有什么抨击体制、反对暴徒，他只是记录问题，没有答案。内在的各种双面元素：身份分裂的拆弹英雄/炸弹狂徒、既是好友又是对手的拍档、大陆骗你女警/帮你女友；外在的各种映射：香港制造危机、黑色口罩横幅、英国留学狂徒、美国超级核弹。高速节奏的套中套下暗涌不断，映后耐人寻味。
, user_name:ZeonGin Sou, comment_time:2020-12-26 03:05:05
comment:
如果说有什么能呈现我去年在香港住了大半年所感受到的一切，那一定是《拆弹专家2》所表现出来的气质。太牛逼了邱礼涛，开场5分钟就把那股无处安放的怒火外化一次超级爆炸，既然大家都不想过了那就把一切都烧成灰。紧接着就是我当时租住的那种小，乱，差的房间，香港人头上是一颗炸弹，脚下又是一颗炸弹，我都惊了。刘德华演的就是香港本身，是善良的，但又是谁把他逼成这样？“我不是疯，那是痛！”“签个名就能改变这个制度吗？”这120分钟的每一分钟我都觉得邱礼涛在另有所指。双方实力你争我夺，想的都只是利用你而已，又有谁在乎他本人？里面用了《水浒传》来代指，那就是“投降派宋江”啊。看似正面的结局，但达成的途径竟然是催眠，植入记忆，欺骗，洗脑。这片子糙肯定还是以往邱礼涛的那种糙，但情绪已经满得喘不过气了。
, user_name:赤之彗星, comment_time:2020-12-25 16:56:48
comment:
今年第一次进影院，很值～看的粤语版。是合格的商业片，情节流畅节奏明快，特效演技都ok。

https://movie.douban.com/subject/30171424/comments?start=260&limit=20&status=P&sort=new_score
comment:
之前看网上说，王晶导演呼吁大家都来看这部片子。看完之后真的想说，王晶导演这次诚不欺我！是真的好精彩的一部港产片！偏执的人就如一个定时炸弹这个点题的比喻用得真好，整部片的紧凑度也真的是爽爆~刘德华50多了啊，那场巷战追逐戏还能又跑又跳~~
, user_name:英年早睡, comment_time:2020-12-24 12:47:02
comment:
看完心情复杂。戴锦华在《返归未来》里分析好莱坞电影和恐怖分子的关系，直接点明前者为后者提供了绝佳范本而非相反，如今视之恐怕亦复如是。细讲起来，那一批判似乎恰恰好是把中国排除在外的，也因此很难不让人有些联想。当然作为一部类型港片，《拆弹专家2》又是成功的，哪怕除去身份的、政治的桑塔格意义上的阐释，它仍然立在今年贫乏的院线顶端。
, user_name:圆首的秘书, comment_time:2020-12-28 19:17:56
comment:
喔，太狂了，太猛了，太狠了，太炸了，惊叹！
, user_name:撇横横竖, comment_time:2020-12-24 18:11:14
comment:
之前看过首映的朋友说这部堪比好莱坞大片，所以每天都盯着它的上映时间，现在第一时间看完电影，果然感觉不负期待啊！用一句话总结刘德华在这部电影里的表演——对这部电影来说，刘德华已经不仅仅只是一个演员那么简单，某些瞬间，我感觉他就是潘乘风本人。
, user_name:金田一二三, comment_time:2020-12-24 09:56:05
comment:
双刘的角色换换多好，刘德华怎么演都不像反社会分子，刘青云倒是越来越阴鸷了。潘乘风浑身上下没有一点像残疾人，整部影片没有涉及半点真实残疾人的生活境况。庞玲和董吵架突然上升到“牺牲一个救一百个”（电车难题），体现了二极管时代的典型思维。“阿风我带你去看医生吧”“我看过了也吃了药”“我是指心理医生”，草，笑得我想吐。19年hk黑暴过去没多久坐在电影院里看这个，能强烈地感受到电影的时效性，不过我总感觉导演有点两面派，骂恐怖分子是为了赚大陆人民的感情💰，影射“com

https://movie.douban.com/subject/30171424/comments?start=320&limit=20&status=P&sort=new_score
comment:
各方面都要比信条好得多，恭喜拆弹专家2获得香港电影金像奖最佳影片
, user_name:rrke, comment_time:2020-12-25 12:43:37
comment:
刘德华的烂演技能好看吗......
, user_name:肥嘟嘟左卫门, comment_time:2020-12-25 07:06:12
comment:
还是香港电影的那一口气拍着电影，到了哪个情绪点就会全身心带入不管不顾了，情绪和情感投入已远远大于在表面把故事人物逻辑编织好这件事了，看到点香港电影快速运镜剪辑在情绪上和精细动作设计上的还魂还是很满足的。
, user_name:西卡里奥, comment_time:2020-12-24 21:26:34
comment:
公映
, user_name:pipboy, comment_time:2020-12-26 21:14:17
comment:
8.0/10【华语片年度最佳】
其实题眼就在当初潘乘风对庞玲说的气话里了：“信不信我造一颗炸弹，然后拆掉它，让全世界都肯定我的能力？”所以在文本上，本片已然做到了类型片的最高阶：立意两不得罪，无所谓谁吃下谁，却偏偏还能有个正向上扬的结局。
诚然，剧作手法上有些笨拙：以人物带故事，却没能用故事点人物。故而后半段两处逻辑链羸弱，马潘两人关系的隐线也没有做好。可当看到黑底白字的横幅竖起，又有谁能无动于衷呢？
这就是华语片中的《黑暗骑士》，是国产类型片饱受桎梏节节败退后，港导为我们留住的最后曙光。
, user_name:眼睛瞪得像铜铃, comment_time:2020-12-24 21:39:40
comment:
场面宏大，剧本也有新意，没有过于明显的硬伤，总体而言值回票价。
, user_name:Tom, comment_time:2020-12-29 11:54:07
comment:
“我给你取名杨过，是叫你知过改过”
, user_name:废话队长, comment_time:2020-12-17 18:08:55
comment:
刘老板一句“我们做拆弹的，哪一次不

https://movie.douban.com/subject/30171424/comments?start=380&limit=20&status=P&sort=new_score
comment:
近年来港片在保持港风剧情一贯优势的前提下，制作越来越精良，机场爆炸片段最让我记忆深刻，火光滔天飞机瞬间化为灰烬效果震撼。真兄弟不会没有争执，女性角色穿插其中很好的调节了紧张的氛围。四星推荐。
, user_name:张霓, comment_time:2020-12-24 15:30:46
comment:
表演略程式化，扣一分，其他构思还挺好的
, user_name:Johny, comment_time:2020-12-25 12:50:08
comment:
酣畅过瘾的爽片
, user_name:荒林, comment_time:2020-12-26 16:49:05
comment:
产生身份焦虑的是刘德华不是谢君豪，只此一点就说明邱礼涛已然决绝地和97世代的香港电影割席。10年来所有的香港商业电影凝望“昨日之辉煌”迟迟不肯撒手，最后在树大招风里摆出风度“让一切随风”。只有邱礼涛凝视着那部凄厉的十年，徒手从血色模糊里挖出愤怒的心肺给陆港共赏。你笑话白发谢君豪是饮血魔头，但是倪妮那句“你信我你就是英雄，不信你就是恐怖分子”才是题眼。整个香港的爆炸都是刘德华自己的左右互搏，但你真的自信倪妮的读心术一生一世不会反伤到自己一分一毫吗。
, user_name:空空儒也, comment_time:2020-12-24 18:49:01
comment:
觉得挺好，但是没什么感觉…可能是因为看的国语配音，也可能是因为onecool调色，总之不是no good，可能这就是low good吧……low good low good也挺好。我给倪妮加一星……🤭
, user_name:海樹, comment_time:2020-12-29 15:56:37
comment:
粤语版好加分！“我是谁”的母题真是百吃不厌。“你是宋江，我是晁盖”这样的台词一出来真的是让观众出一把冷汗。两条男男情感的线路都很抢眼，女性角色的头即使在海报上被放得再大也还是可有可无的甲乙丙丁。
, user_name:子皿与舟, comment_time:2020-12-28 12:09:11
co

https://movie.douban.com/subject/30171424/comments?start=440&limit=20&status=P&sort=new_score
comment:
我与我周旋久
, user_name:lcsun, comment_time:2020-12-26 18:38:54
comment:
太好看了，本来前一天买了票，今天去看之前和男朋友吵架了，但是还是和他去看了因为好想看，结果憋尿好久都没舍得去上厕所，真不戳！
, user_name:好运来, comment_time:2020-12-25 20:52:10
comment:
看到《暗战》的影子，不同点是现在更有钱，冲突也可以升级得更好看。以前看刘德华就是勤奋帅哥而已，现在看来，这种几十年如一日的拼搏自律，体现在角色的脸上行动上很合理，他的神情神态里有着岁月沉淀的自洽，就算是演一个反派，也感觉刘生还是想做好警察，做一个好人！倪妮撑不起，呼吁以后刘生电影里的女友太太都锁定林嘉欣吧！刘青云工具人了，角色吃亏了点。
, user_name:乌苏拉, comment_time:2020-12-28 22:06:34
comment:
用极度快速的节奏，和极度极端的情绪，去掩饰剧情的孱弱
, user_name:朱熠, comment_time:2020-12-27 16:16:16
comment:
虎头豹尾，中段垮掉，不是谁都适合卖腐，节奏很快，但剧情仍有破绽……谢君豪说这一切都是你搞出来的时候，全场爆笑。
, user_name:李棂夏, comment_time:2020-12-27 21:22:22
comment:
帅气
, user_name:尐楓楓最最最最, comment_time:2020-12-24 00:41:27
comment:
被体制“用后即弃”人，在还有利用价值的情况下，通过更改记忆拿回来再利用。
, user_name:Saudade, comment_time:2020-12-26 12:38:25
comment:
不能启用残障人士从事高危工作这个制度本身到底是对还是错？
, user_name:萌 . 李, comment_time:2020-12-26 11:25:26
comment:
剧本没有按照标准化模板来，但场面真的拍的不错

https://movie.douban.com/subject/30171424/comments?start=500&limit=20&status=P&sort=new_score
大约共26页评论


In [125]:
post_data = {
    'name':'18664678368',
    'password':'LJC970412',
    'remember':'false'
}

In [127]:
crawl.add_header({
    "Referer":'https://accounts.douban.com/passport/login'
})


In [128]:
user = crawl.session_post('https://accounts.douban.com/j/mobile/login/basic',data=post_data)

In [129]:
user.text

'{"status":"success","message":"success","description":"处理成功","payload":{"account_info":{"name":"Ljc970412","weixin_binded":false,"phone":"18664678368","avatar":{"medium":"https://img3.doubanio.com\\/icon\\/user_large.jpg","median":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","large":"https://img3.doubanio.com\\/icon\\/user_large.jpg","raw":"https://img3.doubanio.com\\/icon\\/user_large.jpg","small":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","icon":"https://img3.doubanio.com\\/pics\\/icon\\/user_icon.jpg"},"id":"217184556","uid":"217184556"}}}'

In [111]:
params = {
            'q':'紧急救援',
            'cat':''
        }

In [140]:
user.text

'{"status":"success","message":"success","description":"处理成功","payload":{"account_info":{"name":"Ljc970412","weixin_binded":false,"phone":"18664678368","avatar":{"medium":"https://img3.doubanio.com\\/icon\\/user_large.jpg","median":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","large":"https://img3.doubanio.com\\/icon\\/user_large.jpg","raw":"https://img3.doubanio.com\\/icon\\/user_large.jpg","small":"https://img1.doubanio.com\\/icon\\/user_normal.jpg","icon":"https://img3.doubanio.com\\/pics\\/icon\\/user_icon.jpg"},"id":"217184556","uid":"217184556"}}}'

In [133]:
res = dict()

In [134]:
from bs4 import BeautifulSoup

In [137]:
soup = BeautifulSoup(response.content, 'html.parser')
results = soup.find(class_='result-list').find_all(class_='result')
for result in results:
    pic = result.find(class_='pic')
    content = result.find(class_='content')
    img = pic.img.get('src')
    link = content.a.get('href')
    name = content.a.text
    description = content.p.text if content.p else ''
    res[name]={
        'img':img,
        'link':link,
        'description':description
    }

In [145]:
res

{'紧急救援 ': {'img': 'https://img9.doubanio.com/view/subject/s/public/s4562574.jpg',
  'link': 'https://www.douban.com/link2/?url=https%3A%2F%2Fmovie.douban.com%2Fsubject%2F5168928%2F&query=%E7%B4%A7%E6%80%A5%E6%95%91%E6%8F%B4&cat_id=1002&type=search&pos=4',
  'description': '影片描述的是一群紧急救援部队中的爱恨情仇故事。真实感人的剧情。'},
 '紧急救援': {'img': 'https://img1.doubanio.com/view/group/sqxs/public/b41041023a531c8.jpg',
  'link': 'https://www.douban.com/link2/?url=https%3A%2F%2Fwww.douban.com%2Fgroup%2F681165%2F&query=%E7%B4%A7%E6%80%A5%E6%95%91%E6%8F%B4&cat_id=1019&type=search&pos=1',
  'description': '本组是该电影的豆瓣官方讨论小组。\r\n\r\n【本组欢迎】\r\n\r\n1. 个人对电影的理解、感悟和分析；\r\n2. 有价值、有意义的提问；\r\n3. 对电影细节的考据、求证或对背景知识的补充...'},
 '紧急呼救 第一季 ': {'img': 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2568533667.jpg',
  'link': 'https://www.douban.com/link2/?url=https%3A%2F%2Fmovie.douban.com%2Fsubject%2F27043023%2F&query=%E7%B4%A7%E6%80%A5%E6%95%91%E6%8F%B4&cat_id=1002&type=search&pos=5',
  'description': '本剧讲述警察﹑医护人员及消防

In [130]:
a = crawl.session_get('https://www.douban.com/search', params=params)

In [136]:
response = a

In [131]:
a.text

'<!DOCTYPE html>\n<html lang="zh-CN" class="ua-windows ua-ie10">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n  搜索: 紧急救援\n</title>\n    \n  \n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">\n    \n<link rel="search" type="application/opensearchdescription+xml" title="豆瓣搜索" href="/opensearch">\n\n    <script >var _head_start = new Date();</script>\n    <script src="https://img3.doubanio.com/f/shire/72ced6df41d4d158420cebdd254f9562942464e3/js/jquery.min.js"></script>\n    <script src="https://img3.doubanio.com/f/shire/5ecaf46d6954d5a30bc7d99be86ae34031646e00/js/douban.js"></script>\n    <link href="https://img3.doubanio.com/f/shire/859dba5cddc7ed1435808cf5a8ddde5792cd6e0c/css/dou

In [9]:
a = crawl.request_get('http://www.baidu.com',params={'val':1})

In [25]:
def test(*args, **kwargs):
    print(args)
    print(kwargs['ss'])